In [1]:
import pandas as pd
import matplotlib.pyplot as plt
# from matplotlib import rc
import seaborn as sns
import os
import glob
sns.set(style='ticks')
import json
s = json.load( open(r"../custom.json"))  #edit path to json file\n",
plt.rcParams.update(s)

In [2]:
file =r"E:\Documents\Research\Molecular Dynamics\Code\GitHub\Jupyter\Cross Coeff Mie Pot\coeff_nn.csv"
names = []

In [3]:
df = pd.read_csv(file,  skiprows = 0, delimiter =",")


In [4]:
df

,Molecule,Type,Beads,ep/k_b (K),sigma (A),lamda_r
0,Methane,1,1,170.75,3.752,16.39
1,Ethane,2,1,330.25,4.349,27.30
2,Propane,3,1,426.08,4.871,34.29
3,Butane,4,2,256.36,3.961,13.29
4,Pentane,5,2,317.50,4.248,16.06
5,Hexane,6,2,376.35,4.508,19.57
6,Heptane,7,2,436.13,4.766,23.81
7,Octane,8,3,333.70,4.227,16.14
8,Decane,9,3,415.19,4.584,20.92
9,Tetradecane,10,5,363.06,4.183,17.66


In [5]:
Na =6.022*10**23; k = 1.381*10**(-23)

In [6]:
df["ep (kcal/mol)"] = df["ep/k_b (K)"]*Na*k/(4.1858*1000)
# df.loc[11] = ['Graphene',11,0.0,0,3.4,4,0.056]
df=df.sort_values(by=["Type"])
df=df.reset_index()
# df.drop("index")

In [7]:
display(df)

,index,Molecule,Type,Beads,ep/k_b (K),sigma (A),lamda_r,ep (kcal/mol)
0,0,Methane,1,1,170.75,3.752,16.39,0.339248
1,1,Ethane,2,1,330.25,4.349,27.30,0.656143
2,2,Propane,3,1,426.08,4.871,34.29,0.846539
3,3,Butane,4,2,256.36,3.961,13.29,0.509338
4,4,Pentane,5,2,317.50,4.248,16.06,0.630812
5,5,Hexane,6,2,376.35,4.508,19.57,0.747735
6,6,Heptane,7,2,436.13,4.766,23.81,0.866507
7,7,Octane,8,3,333.70,4.227,16.14,0.662998
8,8,Decane,9,3,415.19,4.584,20.92,0.824903
9,9,Tetradecane,10,5,363.06,4.183,17.66,0.721331


In [8]:
def cal(sig_ii,sig_jj,ep_ii, ep_jj,lmd_ii,lmd_jj):
    sig_ij = round((sig_ii+sig_jj)/2,3);
    ep_ij = round(((((sig_ii**3)*sig_jj**3)**(1/2))/sig_ij**3)*(ep_ii*ep_jj)**0.5,3)
    lmd_ij = round(((lmd_ii-3)*(lmd_jj-3))**0.5+3,2) 
    return (sig_ij,ep_ij,lmd_ij)


# lmd_ij = ((lmd_ii-3)*(lmd_jj-3))**0.5+3 

In [9]:
# def ep_ij(ep_ii, ep_jj):
#     cal_ep_ij

In [10]:
# index_mol_i = df.index[df.Molecule =="Methane"][0]
# df.iloc[index_mol_i:]["sigma (A)"]

In [11]:
# pc_df = pd.DataFrame(columns=["i","j","sig_ij"])
print()
# print(mol_list["Methane"].index())
pc_list =[]
for mol_i in df["Molecule"]:
    index_mol_i = df.index[df.Molecule ==mol_i][0]
    for mol_j in df.iloc[index_mol_i:]["Molecule"]:
        loc_i = df.loc[df["Molecule"]==mol_i]
        loc_j = df.loc[df["Molecule"]==mol_j]
        sig_ii = loc_i["sigma (A)"].values[0]
        sig_jj = loc_j["sigma (A)"].values[0]
        ep_ii = loc_i["ep (kcal/mol)"].values[0]
        ep_jj = loc_j["ep (kcal/mol)"].values[0]
        lmd_ii = loc_i["lamda_r"].values[0]
        lmd_jj = loc_j["lamda_r"].values[0]
        
        cross_coeff = cal(sig_ii,sig_jj,ep_ii, ep_jj,lmd_ii,lmd_jj)
        pc_list.append({"i":loc_i["Type"].values[0],"j":loc_j["Type"].values[0],"sig_ij":cross_coeff[0],
                       "ep_ij":cross_coeff[1],"lmd_ij":cross_coeff[2]
                       })
#         print({"i":[loc_i["Type"].values[0]],"j":[loc_j["Type"].values[0]],"sig_ij":[sig_ij(sig_ii,sig_jj)]})
pc_df =pd.DataFrame(pc_list, columns=["pair_coeff","i","j","pot","ep_ij","sig_ij","lmd_ij","lmd_a"])
pc_df["pair_coeff"]="pair_coeff"
pc_df["pot"] ="mie/cut"
pc_df["lmd_a"] =6
pc_df.loc[(pc_df["i"]==11)|(pc_df["j"]==11),"pot"]="lj/cut"
pc_df.loc[(pc_df["pot"]=="lj/cut"),["lmd_ij","lmd_a"]]=""
pc_df.set_index("pair_coeff",inplace=True)
# pc_df.sort_values(by=["i","j"],inplace=True)
pc_df

,i,j,pot,ep_ij,sig_ij,lmd_ij,lmd_a
pair_coeff,,,,,,,
pair_coeff,1,1,mie/cut,0.339,3.752,16.39,6
pair_coeff,1,2,mie/cut,0.468,4.050,21.04,6
pair_coeff,1,3,mie/cut,0.522,4.312,23.47,6
pair_coeff,1,4,mie/cut,0.415,3.856,14.74,6
pair_coeff,1,5,mie/cut,0.460,4.000,16.22,6
pair_coeff,1,6,mie/cut,0.497,4.130,17.9,6
pair_coeff,1,7,mie/cut,0.531,4.259,19.69,6
pair_coeff,1,8,mie/cut,0.472,3.990,16.26,6
pair_coeff,1,9,mie/cut,0.521,4.168,18.49,6


In [12]:
pc_df.to_csv("pot.txt",sep=" ")
# 